# Psql Type Casting with Python (psycopg2)

# <font color=red>Mr Fugu Data Science</font>

# (◕‿◕✿)


# Purpose & Outcome:

+ This is a toolset type of day, you will learn some new tools/skills which WILL come up in your future!

+ Use psycopg2: printing table, columns and schema vs `\d` because it won't directly work
+ Change column datatype
+ Query to find column names for dataframe, instead of hardcoding them in
+ Pattern matching, fuzzy match, levenshtein distance

In [1]:
import pandas as pd
import psycopg2
from psycopg2 import sql # format sql queries with params

# Import the 'config' function from the config_user_dta.py file:
from config_user_dta import config  # call my user credentials

`---------------------------------`

If you do not want to create init or config files: do something similar to this

**`import psycopg2
conn = psycopg2.connect("dbname=test user=postgres") # Connect to an existing database
cur = conn.cursor() #Open a cursor to perform database operations`**

In [2]:
# Establish a connection to the database by creating a cursor object

# Get the config params
params_ = config()

# Connect to the Postgres_DB:
conn = psycopg2.connect(**params_)

# Create new_cursor allowing us to write Python to execute PSQL:
cur = conn.cursor()

conn.autocommit = True  # read documentation understanding when to Use & NOT use (TRUE)

# psycopg2.extras.register_hstore(conn)

In [109]:
# our files:

car_dlr_one=pd.read_csv('cardealer_add_deduct_list.csv')

car_dlr_two=pd.read_csv('car_dealer_firstpart.csv')

car_dlr_one.head()
car_dlr_two.columns


,publish_date,vin,make,model,model_year
0,8/28/2020,102280487,Kia,Soul,2019
1,8/28/2020,102298407,Chevrolet,Malibu,2017
2,8/28/2020,102633593,Ford,Fusion,2017


In [ ]:
# For future endeavor: figure out how to create a table dynamically

In [ ]:
# convert_dtypes=[]
# for i in car_dlr_one.iteritems():
#     if i[1].dtype=='object':
#         convert_dtypes.append([i[0],'text'])

#     elif re.findall(r'\b(int).+',str(i[1])):
#         convert_dtypes.append([i[0],         
#                                re.findall(r'\b(int).+',str(i[1]))[0]])
        
# for i in convert_dtypes:
#     print(i)
# #     find_dta('prac',i[0],i[1])
# # for i in convert_dtypes:
# # convert_dtypes

In [ ]:
# This is annoying to create like this every time I want to figure out a dynamic way in 
# the future! Avoid hard coding etc....

In [94]:
def create_staging_table(cursor):
    cursor.execute("""
        DROP TABLE IF EXISTS car_dealer_01;
        CREATE UNLOGGED TABLE car_dealer_01 (
            vin                varchar(30),
            idx                int,
            model_number_list  text
            
        );""")
    
# Send the Schema to PSQL

with conn.cursor() as cursor:
    create_staging_table(cursor)

In [153]:
def create_staging_table(cursor):
    cursor.execute("""
        DROP TABLE IF EXISTS car_dealer_02;
        CREATE UNLOGGED TABLE car_dealer_02 (
            publish_date       date,
            vin                varchar(30),
            make               varchar(30),
            model              varchar(30),               
            model_year         int 
        );""")
    
# Send the Schema to PSQL
with conn.cursor() as cursor:
    create_staging_table(cursor)

# Find Table Schema:

In [154]:
ss="""
SELECT table_name, 
column_name, 
data_type 
FROM information_schema.columns 
WHERE table_name={table_name} ;
"""

# we are using: literal because the table_name will call our table and we need str repr.
o=sql.SQL(ss).format(table_name=sql.Literal('car_dealer_02'))
cur.execute(o)
cur.fetchall()

[('car_dealer_02', 'publish_date', 'date'),
 ('car_dealer_02', 'vin', 'character varying'),
 ('car_dealer_02', 'make', 'character varying'),
 ('car_dealer_02', 'model', 'character varying'),
 ('car_dealer_02', 'model_year', 'integer')]

In [74]:
# insert data into table:
car_dlr_subset_01=car_dlr_one.loc[:,['idx','vin','model_number_list']]
car_dlr_subset_01.values

array([[0, 102280487, "['B2522']"],
       [0, 102280487, "['B2522']"],
       [1, 102298407, "['1ZD69']"],
       [1, 102298407, "['1ZD69']"],
       [1, 102298407, "['1ZD69']"],
       [1, 102298407, "['1ZD69']"],
       [2, 102633593, "['P0H']"],
       [2, 102633593, "['P0H']"],
       [2, 102633593, "['P0H']"],
       [2, 102633593, "['P0H']"],
       [2, 102633593, "['P0H']"],
       [2, 102633593, "['P0H']"]], dtype=object)

# Send Dataframe:

In [102]:
from psycopg2 import extras


def fcn(df,table,cur):
#     df=addr_df_
#     table='staging_fake_ppl02'
    # df is the dataframe
    if len(df) > 0:
        df_columns = list(df)
        # create (col1,col2,...)
        columns = ",".join(df_columns)

        # create VALUES('%s', '%s",...) one '%s' per column
        values = "VALUES({})".format(",".join(["%s" for _ in df_columns])) 

        #create INSERT INTO table (columns) VALUES('%s',...)
        insert_stmt = "INSERT INTO {} ({}) {}".format(table,columns,values)
        cur.execute("truncate " + table + ";")  #avoiding uploading duplicate data!
        cur = conn.cursor()
        psycopg2.extras.execute_batch(cur, insert_stmt, df.values)
    conn.commit()

In [114]:
# car_dlr_subset_01
# fcn(car_dlr_subset_01,'car_dealer_01',cur)

cur.execute("select * from car_dealer_01 limit 4")
cur.fetchall()

[('102280487', 0, "['B2522']"),
 ('102280487', 0, "['B2522']"),
 ('102298407', 1, "['1ZD69']"),
 ('102298407', 1, "['1ZD69']")]

In [171]:
car_dlr_subset02=car_dlr_two.loc[:,['publish_date','vin','make','model','model_year']]

fcn(car_dlr_subset02,'car_dealer_02',cur)

cur.execute("select * from car_dealer_02 limit 4")
cur.fetchall()


[(datetime.date(2020, 8, 28), 102280487, 'Kia', 'Soul', 2019),
 (datetime.date(2020, 8, 28), 102298407, 'Chevrolet', 'Malibu', 2017),
 (datetime.date(2020, 8, 28), 102633593, 'Ford', 'Fusion', 2017)]

# `Convert Column data-type:`

+ convert the `Vin` column form `varchar to integer`

In [174]:

ww="""
ALTER TABLE car_dealer_02 ALTER COLUMN
vin TYPE integer 
USING (vin::integer);
"""
cur.execute(ww)

In [175]:
ss="""
SELECT table_name, 
column_name, 
data_type 
FROM information_schema.columns 
WHERE table_name={table_name} ;
"""

# we are using: literal because the table_name will call our table and we need str repr.
o=sql.SQL(ss).format(table_name=sql.Literal('car_dealer_02'))
cur.execute(o)
cur.fetchall()

[('car_dealer_02', 'publish_date', 'date'),
 ('car_dealer_02', 'vin', 'integer'),
 ('car_dealer_02', 'make', 'character varying'),
 ('car_dealer_02', 'model', 'character varying'),
 ('car_dealer_02', 'model_year', 'integer')]

# `Mogrify:`

If you end up having frequent queries as a daily routine: but, can't trace back from your server what code may be giving you issues then `mogrify` can come to the rescue for debugging. 

In [178]:
# good for error checking or how exactly psql are interpreting what you input
print(cur.mogrify(o))
print('-------------------')
print(cur.mogrify(ww))

b"\nSELECT table_name, \ncolumn_name, \ndata_type \nFROM information_schema.columns \nWHERE table_name='car_dealer_02' ;\n"
-------------------
b'\nALTER TABLE car_dealer_02 ALTER COLUMN\nvin TYPE integer \nUSING (vin::integer);\n'


# This can be beneficial for your workflow for instance: 

+ Because, you can incorporate this after your query and before you use the cursor.execute( )

+ cursor.execute( ) is preventing you from seeing what query was actually sent if you had a problem during the way.

Such as 

`sql_="""select * from my_table where ....;"""
some_name=cursor.mogrify(sql_)
cur.execute(some_name)`

# Fuzzy Match:

`-------------------------`

# `Trigram:`

+ This extension `pg_trgm` will allow some fuzzy string matching

# **`Ex 1. )`**  Find a person but you don't remember the name, only that it is close to `Aldo`

+ Consider if you had an idea but, frankly couldn't remember the name. It happens to us all and this will help you in a pinch.

`You are going over a specific column to find them`

In [190]:

f='''
CREATE EXTENSION pg_trgm;
SELECT 
*
FROM staging_fake_ppl
ORDER BY SIMILARITY(first_name,'Aldo') DESC
LIMIT 5;

'''
cur.execute(f)
cur.fetchall()

# cur.execute('''Drop extension pg_trgm ''')


# There is a great deal of function with this please read the link below!

# https://www.postgresql.org/docs/current/pgtrgm.html (examples)

[('3118-3897-1674-7996', 'ajfeqi@iufeb.efk', 'Alfredo', 'Munoz', '5244214267'),
 ('9997-7751-7665-6525', 'hfl@bnkxsvp.ufq', 'Ali', 'Borrego', '2493418317'),
 ('1806-2704-9336-8126', 'swimxj@pjboqc.fow', 'Ali', 'Kouns', '4218255468'),
 ('2117-2588-7985-4527', 'dxbmswr@o.mlb', 'Alec', 'Johnson', '4259785921'),
 ('1758-6471-8929-2781', 'ruft@ip.drj', 'Alex', 'Acosta', '8728626214')]

# **`Ex. 2 )`** Let's assume you needed some confidence

+ `this is from 0-1, the lower the number; the less specific you want!`

*default is 0.3*

In [197]:
conf_str='''
SELECT
*
FROM staging_fake_ppl
WHERE SIMILARITY(last_name,'Bor') > 0.4 ;

'''
cur.execute(conf_str)
cur.fetchall()

[('9538-4639-5634-2281', 'cvjkmi@t.cko', 'Cailinanne', 'Borja', '2968327286'),
 ('2285-2386-1324-5043', 'idv@pqdvscle.owa', 'Gunnar', 'Borda', '2595211562'),
 ('2731-8907-1044-6831', 'dstcjoaxw@uer.yef', 'Eric', 'Borom', '5864561645')]

# `Phonetic Spelling`:

+ Using the **`fuzzystrmatch`** extension will convert your string into four letter codes to search for codes that match. An example: `flour and flower` are the same code.

+ `Side NOTE`: NON-English names will have issues with finding matches

# `Ex. )`

In [207]:
phone_='''
CREATE EXTENSION fuzzystrmatch;

SELECT
*
FROM staging_fake_ppl WHERE
SOUNDEX(first_name) = SOUNDEX('Dony A');
'''
cur.execute(phone_)
cur.fetchall()

# Can Drop the extension if not needed anymore: uncomment below

# cur.execute('''drop extension fuzzystrmatch''')

[('3388-9288-1720-9622', 'ysbe@ldajvymbqu.nga', 'Diana', 'Song', '6212544297'),
 ('6496-8623-1961-2916',
  'ktoey@vkqwexc.lbh',
  'Diana',
  'Olvedo Enriquez',
  '5234813586'),
 ('2725-5432-4694-7135', 'ivlm@wbsaim.lnj', 'Don', 'Martinez', '4657397958'),
 ('1736-4079-2112-1254', 'h@axlkjm.bip', 'Doan', 'Doht', '9416178251'),
 ('3929-1702-5794-9652', 'kofshc@cquxn.zxn', 'Deion', 'Holm', '4758758462'),
 ('3970-4084-4226-8551', 'fiw@uhfndrk.ugb', 'Danny', 'John', '2976273517'),
 ('7592-3801-8837-8387', 'o@bodzfaxin.dtg', 'Deanna', 'Newton', '7311849615'),
 ('7749-8107-5618-3026', 'ory@vukzhy.tnb', 'Danny', 'el-Zadeh', '6394522678'),
 ('9134-3311-6532-5063',
  'ynfhstlkwi@koayhrmx.kvd',
  'Danny',
  'Snively',
  '3174658291'),
 ('5067-7345-8241-7248',
  'ifshounla@mbkjcvewo.ezt',
  'Deeana',
  'Crespin',
  '6149173275'),
 ('9202-6310-9015-7438', 'cwxmht@luh.sft', 'Danni', 'Stahl', '9484396785'),
 ('2762-8722-9883-9155', 'c@rxtzgsemc.mnt', 'Deion', 'Begay', '4921596928'),
 ('9231-7927-1920-

# Levenshtein Distance:

+ Think of edit distance, between strings at a character level.
    + This is measured by deletes, inserts and inserts
+ ex.) Convert `long` into `length`
   
we would have to change the `o` to `e` by 1 insert, then add `t` and `h` that would be a distance of 4.

In [213]:
levi_='''
SELECT
*,
LEVENSHTEIN(first_name, 'Dony w')
FROM staging_fake_ppl
ORDER BY LEVENSHTEIN(first_name, 'Dony w') ASC
LIMIT 5
'''

cur.execute(levi_)
cur.fetchall()

[('2958-5980-8949-4378', 'lrnyj@ianwzptm.vsy', 'Tony', 'Wu', '3948634793', 3),
 ('8966-6746-2369-5926', 'ln@xsjv.gsq', 'Rony', 'al-Taha', '5236191587', 3),
 ('2948-8505-3391-2567',
  'qoxdjanvfp@xqekfs.pqi',
  'Dondre',
  'Jaure',
  '9589588163',
  3),
 ('2725-5432-4694-7135',
  'ivlm@wbsaim.lnj',
  'Don',
  'Martinez',
  '4657397958',
  3),
 ('9231-7927-1920-8835',
  'mlotjsuy@asy.jou',
  'Don',
  "O'Leary",
  '7567296748',
  3)]

# `Convert String Representation:`

In [215]:
# Convert to lowercase()

lwrcs='''
select first_name,lower(first_name) from staging_fake_ppl limit 4

'''

cur.execute(lwrcs)
pd.DataFrame(cur.fetchall(),columns=['Orginal_FirstName','Converted'])

,Orginal_FirstName,Converted
0,Donyell Ann,donyell ann
1,Bishop,bishop
2,Connor,connor
3,Kylie,kylie


# `Pattern Matching:`

`----------------------------------------------`

+ If using `regex` look into how this may affect speed


# `Ex.1 )`: using `regex` to find first_names that are either starting with J or G

In [220]:
reg_mx='''
select * from staging_fake_ppl where first_name ~'^(J|G).*'
limit 5
'''
cur.execute(reg_mx)
cur.fetchall()

[('5909-1054-2911-2100',
  'nauvojf@tz.mjc',
  'Justin',
  'al-Yousif',
  '9214512756'),
 ('5027-1684-3271-9573',
  'esculkhwaj@lgime.kpd',
  'Justin',
  'Vigil',
  '4639124827'),
 ('4266-4647-9544-7225', 'bkzy@ozixylp.ixt', 'Jade', 'el-Qadir', '4121832691'),
 ('5737-9277-9259-1227',
  'vpzcje@jhtnqoaire.uag',
  'Juan',
  'Afuang',
  '8574182659'),
 ('6447-6210-9155-9438',
  'lwysbtujna@tsrzhyjm.wyp',
  'Joshua',
  'Brown',
  '3571528429')]

# `Ex 1A. )` Other Regex example

find vin numbers that match `102` and convert it to text

In [236]:
reg_mx01='''
select * from car_dealer_02 where 
CAST(vin AS TEXT) LIKE '102%'

'''
cur.execute(reg_mx01)
cur.fetchall()

[(datetime.date(2020, 8, 28), 102280487, 'Kia', 'Soul', 2019),
 (datetime.date(2020, 8, 28), 102298407, 'Chevrolet', 'Malibu', 2017),
 (datetime.date(2020, 8, 28), 102633593, 'Ford', 'Fusion', 2017)]

# `Ex.2 ):` Using LIKE/ILIKE 

+ This allows you to search for something that you have a clue but not exactly with either case or case-insensitive

+ `%` operator will match zero or more occurences
+ `_` the underscore will match single characters

In [223]:
lk_='''
select * from staging_fake_ppl where last_name like 'Bega%'

'''
cur.execute(lk_)
cur.fetchall()

[('1270-8146-6857-9420',
  'pdw@uwarlpyvgj.unh',
  'Umaima',
  'Begay',
  '5129433167'),
 ('6192-4667-4124-8778',
  'woefqdgtmv@pqmxzscfg.svu',
  'Theophilus',
  'Begay',
  '6286151359'),
 ('7927-4545-1401-6815', 'hqvixtof@nj.ntf', 'Jordyn', 'Begaye', '1258755914'),
 ('2762-8722-9883-9155', 'c@rxtzgsemc.mnt', 'Deion', 'Begay', '4921596928'),
 ('8324-4327-8557-6815',
  'tjerqvzb@fqlbmdh.zbu',
  'Munjid',
  'Begay',
  '5197293945'),
 ('6840-3083-3722-6802', 'ldmvn@wfriybh.hvm', 'Lauren', 'Begay', '5965799654'),
 ('9484-4433-3524-3530',
  'qkvjhybgod@igqnlwt.bxu',
  'Keisha',
  'Begaye',
  '6797265431'),
 ('7767-6352-2840-2471', 'layb@vasnk.obs', 'Gabriella', 'Begay', '7939216827'),
 ('2325-6986-1990-7780',
  'qhdlknupjg@fy.xgd',
  'Sarah',
  'Begay-Watson',
  '9634151524'),
 ('8972-6928-5770-7435',
  'ijgtoa@mxjazrtgdw.jzs',
  'Juan',
  'Begay',
  '9347197248'),
 ('6171-5824-2335-6706', 'ryai@ngr.kbf', 'Ahlaam', 'Begay', '2831655962')]

# `Ex.2 )` Using the underscore

+ This will match anything that has `ega` and any letters in from of it will the use of `_`

In [228]:
lk_='''
select * from staging_fake_ppl where last_name like '_ega%'

'''
cur.execute(lk_)
df_=cur.fetchall()
             
# Get the Column Names:
cur.execute("Select * FROM staging_fake_ppl LIMIT 0")
colnames = [desc[0] for desc in cur.description]

pd.DataFrame(df_,columns=colnames).head()

,credit_card,email,first_name,last_name,primary_phone
0,1270-8146-6857-9420,pdw@uwarlpyvgj.unh,Umaima,Begay,5129433167
1,8233-3553-7350-4143,aqucnfhz@vtef.vcu,Santee,Vega,4362355976
2,6192-4667-4124-8778,woefqdgtmv@pqmxzscfg.svu,Theophilus,Begay,6286151359
3,7927-4545-1401-6815,hqvixtof@nj.ntf,Jordyn,Begaye,1258755914
4,2762-8722-9883-9155,c@rxtzgsemc.mnt,Deion,Begay,4921596928


# <font color=red>Like</font>, Share &

# <font color=red>SUB</font>scribe

# Citations & Help:


# <font size=7>◔̯◔</font>



`psql shorcuts`

https://www.postgresqltutorial.com/psql-commands/


`Casting`

https://www.postgresqltutorial.com/postgresql-cast/

https://stackoverflow.com/questions/56260366/insert-into-table-with-type-casting-and-condition-in-insert-statement

https://joshuaotwell.com/copy-and-cast-bulk-uploads-in-postgresql/

`Mogrify`

https://www.compose.com/articles/formatted-sql-in-python-with-psycopgs-mogrify/

`Pattern Matching`

https://www.postgresql.org/docs/9.3/functions-matching.html


`Fuzzy Match`

https://www.freecodecamp.org/news/fuzzy-string-matching-with-postgresql/


`Function for schema columns convert df`

https://www.regular-expressions.info/wordboundaries.html

https://www.postgresql.org/docs/9.5/datatype.html

http://www.postgis.us/presentations/PGOpen2018_data_loading.pdf

https://www.geeksforgeeks.org/python-ways-to-print-list-without-quotes/

`psycopg2.sql stuff for future`

http://devdoc.net/python/psycopg2-doc-2.7.3.2/sql.html

https://stackoverflow.com/questions/56529928/psycopg2-how-to-dynamically-create-tables-insert-values-into-postgresql-db-fr

https://stackoverflow.com/questions/54714576/format-create-table-query-with-list-of-identifiers

https://www.postgresql-archive.org/Creating-dynamically-typed-tables-using-psycopg2-s-built-in-formatting-td6090126.html

`misc`

https://stackoverflow.com/questions/10252247/how-do-i-get-a-list-of-column-names-from-a-psycopg2-cursor